#### 1. скачать набор данных маркетинговых кампаний отсюда https://www.kaggle.com/davinwijaya/customer-retention

In [32]:
import numpy as np
import pandas as pd

from IPython.display import Image

from sklearn.model_selection import train_test_split

!pip install scikit-uplift==0.2.0

from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel, ClassTransformation, TwoModels

!pip install causalml

from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

from catboost import CatBoostClassifier


%matplotlib inline

  Using cached xgboost-1.5.2-py3-none-win_amd64.whl (106.6 MB)
Processing c:\users\alexa\appdata\local\pip\cache\wheels\fe\cd\78\a7e873cc049759194f8271f780640cf96b35e5a48bef0e2f36\pydotplus-2.0.2-py3-none-any.whl
  Using cached shap-0.37.0-cp38-cp38-win_amd64.whl (377 kB)
  Using cached dill-0.3.4-py2.py3-none-any.whl (86 kB)
  Using cached pygam-0.8.0-py2.py3-none-any.whl (1.8 MB)
  Using cached torch-1.10.1-cp38-cp38-win_amd64.whl (226.6 MB)
  Using cached pyro_ppl-1.8.0-py3-none-any.whl (713 kB)
  Using cached slicer-0.0.3-py3-none-any.whl (11 kB)
  Using cached progressbar2-4.0.0-py2.py3-none-any.whl (26 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached python_utils-3.1.0-py2.py3-none-any.whl (19 kB)
  Running setup.py clean for causalml
Failed to build causalml
    Running setup.py install for causalml: started
    Running setup.py install for causalml: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\alexa\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\alexa\\AppData\\Local\\Temp\\pip-install-r9b3u_oi\\causalml\\setup.py'"'"'; __file__='"'"'C:\\Users\\alexa\\AppData\\Local\\Temp\\pip-install-r9b3u_oi\\causalml\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\alexa\AppData\Local\Temp\pip-wheel-brwsaxx4'
       cwd: C:\Users\alexa\AppData\Local\Temp\pip-install-r9b3u_oi\causalml\
  Complete output (78 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\causalml
  copying causalml\features.py -> build\lib.win-amd64-3.8\causalml
  copying causalml\match.py -> build\lib.win-amd64-3.8\causalml
  copying causalml\propensity.py

ModuleNotFoundError: No module named 'causalml.inference'

In [37]:
df = pd.read_csv('data.csv', delimiter=',')
df.info()

df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64000 entries, 0 to 63999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   recency        64000 non-null  int64  
 1   history        64000 non-null  float64
 2   used_discount  64000 non-null  int64  
 3   used_bogo      64000 non-null  int64  
 4   zip_code       64000 non-null  object 
 5   is_referral    64000 non-null  int64  
 6   channel        64000 non-null  object 
 7   offer          64000 non-null  object 
 8   conversion     64000 non-null  int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 4.4+ MB


,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,offer,conversion
0,10,142.44,1,0,Surburban,0,Phone,Buy One Get One,0
1,6,329.08,1,1,Rural,1,Web,No Offer,0
2,7,180.65,0,1,Surburban,1,Web,Buy One Get One,0
3,9,675.83,1,0,Rural,1,Web,Discount,0
4,2,45.34,1,0,Urban,0,Web,Buy One Get One,0


In [38]:
df['conversion'].value_counts()
df['offer'].value_counts()

Buy One Get One    21387
Discount           21307
No Offer           21306
Name: offer, dtype: int64

#### 2. там поле conversion - это целевая переменная, а offer - коммуникация. Переименовать поля (conversion -> target, offer -> treatment) и привести поле treatment к бинарному виду (1 или 0, т.е было какое-то предложение или нет) - значение No Offer означает отсутствие коммуникации, а все остальные - наличие.

In [39]:
# переименовываем

df = df.rename(columns={'conversion': 'target', 'offer': 'treatment'})

Приводим в бинарный вид и удаляем признак No Offer - он не дает никакой дополнительной информации, т.к. нули в 2-х других признаках автоматически означают No Offer

In [40]:
df.loc[df['treatment'] != 'No Offer', 'treatment'] = 1
df.loc[df['treatment'] == 'No Offer', 'treatment'] = 0
df['treatment'] = df['treatment'].astype(np.uint8)

df.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,target
0,10,142.44,1,0,Surburban,0,Phone,1,0
1,6,329.08,1,1,Rural,1,Web,0,0
2,7,180.65,0,1,Surburban,1,Web,1,0
3,9,675.83,1,0,Rural,1,Web,1,0
4,2,45.34,1,0,Urban,0,Web,1,0


#### 3. сделать разбиение набора данных не тренировочную и тестовую выборки

In [41]:
X_train, X_test, y_train, y_test, treat_train, treat_test = train_test_split(df.drop(columns=['target']), 
                                                            df['target'],
                                                            df['treatment'],
                                                            random_state=0)

X_train.shape, X_test.shape, y_train.shape, y_test.shape, treat_train.shape

((48000, 8), (16000, 8), (48000,), (16000,), (48000,))

#### 4.сделать feature engineering на ваше усмотрение (допускается свобода выбора методов)

In [42]:
# Рaссмотрим пользователей по сумме их покупок для разделения.

df['history'].describe()


count    64000.000000
mean       242.085656
std        256.158608
min         29.990000
25%         64.660000
50%        158.110000
75%        325.657500
max       3345.930000
Name: history, dtype: float64

In [43]:
# Пусть юзеры с заказами на сумму >= 1000 будут китами, на сумму 200-1000 - дельфинами, и до 200 - пескарями.

def segment_customers(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df['Whales'] = 0
    df['Dolphins'] = 0
    df['Minnows'] = 0
    df.loc[df['history'] >= 1000, 'Whales'] = 1
    df.loc[(df['history'] < 1000) & (df['history'] >= 200), 'Dolphins'] = 1
    df.loc[df['history'] < 200, 'Minnows'] = 1
    return df

X_train = segment_customers(X_train)
X_test = segment_customers(X_test)

X_train.head()

,recency,history,used_discount,used_bogo,zip_code,is_referral,channel,treatment,Whales,Dolphins,Minnows
1098,8,63.58,1,0,Surburban,1,Phone,1,0,0,1
13764,3,395.35,1,0,Surburban,1,Web,1,0,1,0
45116,4,1307.99,1,1,Rural,1,Phone,1,1,0,0
15363,10,159.01,0,1,Surburban,1,Web,0,0,0,1
44498,1,276.00,1,0,Urban,0,Phone,0,0,1,0


#### 5. провести uplift-моделирование 3 способами: одна модель с признаком коммуникации (S learner), модель с трансформацией таргета (трансформация классов п. 2. 1) и вариант с двумя независимыми моделями

In [44]:
# Сделаем датафрейм для сравнения метрик:

metrics_df = pd.DataFrame(columns=['uplift@10%', 'uplift@20%'])

In [45]:
# Категориальные признаки:

cat_features = ['zip_code', 'channel']

#### S-leaner

In [46]:
from sklift.metrics import uplift_at_k
from sklift.viz import plot_uplift_preds
from sklift.models import SoloModel

# sklift поддерживает любые модели, 
# которые удовлетворяют соглашениями scikit-learn
# Для примера воспользуемся catboost
from catboost import CatBoostClassifier

sm = SoloModel(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
sm = sm.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_sm = sm.predict(X_test)
print(uplift_sm)

sm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.1)
sm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_sm, treatment=treat_test, strategy='by_group', k=0.2)


[0.06326457 0.06920032 0.07275016 ... 0.06685124 0.02610939 0.07757569]


In [47]:
# Сохраним метрики модели

metrics_df = metrics_df.append({'uplift@10%': sm_score_10, 'uplift@20%': sm_score_20}, ignore_index=True)

#### Transform Learner

In [48]:
ct = ClassTransformation(CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True))
ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})

uplift_ct = ct.predict(X_test)
print(uplift_ct)

ct_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.1)
ct_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_ct, treatment=treat_test, strategy='by_group', k=0.2)

<ipython-input-48-70bd006fd285>:2: UserWarning: It is recommended to use this approach on treatment balanced data. Current sample size is unbalanced.
  ct = ct.fit(X_train, y_train, treat_train, estimator_fit_params={'cat_features': cat_features})


[-0.68133514 -0.66582944 -0.61137839 ...  0.78141424 -0.65206333
 -0.32893603]


In [50]:
# Сохраняем метрики модели:

metrics_df = metrics_df.append({'uplift@10%': ct_score_10, 'uplift@20%': ct_score_20}, ignore_index=True)

#### Two Model Learner

In [51]:
tm = TwoModels(
    estimator_trmnt=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    estimator_ctrl=CatBoostClassifier(iterations=20, thread_count=2, random_state=42, silent=True), 
    method='vanilla'
)
tm = tm.fit(
    X_train, y_train, treat_train,
    estimator_trmnt_fit_params={'cat_features': cat_features}, 
    estimator_ctrl_fit_params={'cat_features': cat_features}
)

uplift_tm = tm.predict(X_test)
print(uplift_tm)

tm_score_10 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.1)
tm_score_20 = uplift_at_k(y_true=y_test, uplift=uplift_tm, treatment=treat_test, strategy='by_group', k=0.2)

[0.08574949 0.08029174 0.09133908 ... 0.07039956 0.0432995  0.12032493]


In [52]:
# Сохраняем:

metrics_df = metrics_df.append({'uplift@10%': tm_score_10, 'uplift@20%': tm_score_20}, ignore_index=True)

#### 6. вывести единую таблицу сравнения метрик uplift@10%, uplift@20% этих 3 моделей

In [53]:
metrics_df

,uplift@10%,uplift@20%
0,0.059370,0.068407
1,0.225533,0.195837
2,0.079349,0.065813


#### 7. построить модель UpliftTreeClassifier и попытаться описать словами полученное дерево

In [54]:
def transform_data_to_tree(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df = pd.get_dummies(df, drop_first=True)
    return df

In [55]:
X_train_tree = transform_data_to_tree(X_train)
X_test_tree = transform_data_to_tree(X_test)
features = [col for col in X_train_tree]

In [57]:
!git clone https://github.com/uber/causalml.git
%cd causalml
!pip install -r requirements.txt
!python setup.py build_ext --inplace
!python setup.py install

C:\Users\alexa\causalml


fatal: destination path 'causalml' already exists and is not an empty directory.


running build_ext
building 'causalml.inference.tree.causaltree' extension


error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/


running install
running bdist_egg
running egg_info
writing causalml.egg-info\PKG-INFO
writing dependency_links to causalml.egg-info\dependency_links.txt
writing requirements to causalml.egg-info\requires.txt
writing top-level names to causalml.egg-info\top_level.txt
reading manifest file 'causalml.egg-info\SOURCES.txt'
reading manifest template 'MANIFEST.in'
writing manifest file 'causalml.egg-info\SOURCES.txt'
installing library code to build\bdist.win-amd64\egg
running install_lib
running build_py
running build_ext
building 'causalml.inference.tree.causaltree' extension


error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/


In [58]:
%%time
from IPython.display import Image
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from causalml.inference.tree import uplift_tree_string, uplift_tree_plot

uplift_model = UpliftTreeClassifier(max_depth=8, min_samples_leaf=200, min_samples_treatment=50,
                                    n_reg=100, evaluationFunction='KL', control_name='control')

uplift_model.fit(X_train_tree.values,
                 treatment=treat_train.map({1: 'treatment1', 0: 'control'}).values,
                 y=y_train)

graph = uplift_tree_plot(uplift_model.fitted_uplift_tree, features)
Image(graph.create_png())

ModuleNotFoundError: No module named 'causalml.inference'